# [국민 건강검진 데이터 AI 모델링] - LDL 콜레스테롤 수치 예측

> (가설) 시력, 청력, 치아 관련 문항은 콜레스테롤과 관련없음





In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('건강검진정보.csv', encoding='cp949')

In [3]:
df

,HCHK_YEAR,IDV_ID,SIDO,SEX,AGE_GROUP,HEIGHT,WEIGHT,WAIST,SIGHT_LEFT,SIGHT_RIGHT,...,GAMMA_GTP,SMK_STAT_TYPE_CD,DRK_YN,HCHK_OE_INSPEC_YN,CRS_YN,TTH_MSS_YN,ODT_TRB_YN,WSDM_DIS_YN,TTR_YN,DATA_STD_DT
0,2021,4770160,47,1,16,165,55,80.0,0.0,0.0,...,29.0,1.0,0.0,0,NaN,NaN,NaN,NaN,NaN,2022-08-11
1,2021,3065806,41,1,10,180,80,84.0,1.0,0.0,...,22.0,3.0,1.0,0,NaN,NaN,NaN,NaN,NaN,2022-08-11
2,2021,2673031,11,1,16,150,35,64.0,0.0,0.0,...,36.0,2.0,0.0,0,NaN,NaN,NaN,NaN,NaN,2022-08-11
3,2021,2025357,11,1,16,165,60,84.0,0.0,0.0,...,16.0,3.0,1.0,0,NaN,NaN,NaN,NaN,NaN,2022-08-11
4,2021,1976052,41,2,9,160,50,76.0,1.0,1.0,...,19.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN,2022-08-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425529,2021,4447892,11,2,14,150,60,91.0,0.0,0.0,...,32.0,1.0,0.0,1,0.0,NaN,NaN,NaN,0.0,2022-08-11
425530,2021,2763417,47,1,9,170,100,99.0,1.0,1.0,...,73.0,1.0,0.0,0,NaN,NaN,NaN,NaN,NaN,2022-08-11
425531,2021,572436,47,2,10,150,75,98.0,0.0,0.0,...,70.0,1.0,0.0,0,NaN,NaN,NaN,NaN,NaN,2022-08-11
425532,2021,541281,11,2,10,160,55,70.0,0.0,0.0,...,140.0,1.0,0.0,1,0.0,NaN,NaN,NaN,0.0,2022-08-11


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425534 entries, 0 to 425533
Data columns (total 34 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   HCHK_YEAR          425534 non-null  int64  
 1   IDV_ID             425534 non-null  int64  
 2   SIDO               425534 non-null  int64  
 3   SEX                425534 non-null  int64  
 4   AGE_GROUP          425534 non-null  int64  
 5   HEIGHT             425534 non-null  int64  
 6   WEIGHT             425534 non-null  int64  
 7   WAIST              425499 non-null  float64
 8   SIGHT_LEFT         425438 non-null  float64
 9   SIGHT_RIGHT        425442 non-null  float64
 10  HEAR_LEFT          425446 non-null  float64
 11  HEAR_RIGHT         425447 non-null  float64
 12  BP_HIGH            425526 non-null  float64
 13  BP_LWST            425525 non-null  float64
 14  BLDS               425528 non-null  float64
 15  TOT_CHOLE          425058 non-null  float64
 16  TR

## **전처리**

HCHK_OE_INSPEC_YN: 구강검진 수검여부

CRS_YN: 치아우식증 유무

TTH_MSS_YN, ODT_TRB_YN, WSDM_DIS_YN, TTR_YN: 치아 관련

In [5]:
# 데이터 제거 (가설 참고하기)
df.drop(['SIGHT_LEFT', 'SIGHT_RIGHT', 'HEAR_LEFT', 'HEAR_RIGHT', 'HCHK_OE_INSPEC_YN', 'CRS_YN', 'TTH_MSS_YN', 'ODT_TRB_YN', 'WSDM_DIS_YN', 'TTR_YN'], axis=1, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425534 entries, 0 to 425533
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HCHK_YEAR         425534 non-null  int64  
 1   IDV_ID            425534 non-null  int64  
 2   SIDO              425534 non-null  int64  
 3   SEX               425534 non-null  int64  
 4   AGE_GROUP         425534 non-null  int64  
 5   HEIGHT            425534 non-null  int64  
 6   WEIGHT            425534 non-null  int64  
 7   WAIST             425499 non-null  float64
 8   BP_HIGH           425526 non-null  float64
 9   BP_LWST           425525 non-null  float64
 10  BLDS              425528 non-null  float64
 11  TOT_CHOLE         425058 non-null  float64
 12  TRIGLYCERIDE      425054 non-null  float64
 13  HDL_CHOLE         425054 non-null  float64
 14  LDL_CHOLE         417431 non-null  float64
 15  HMG               425529 non-null  float64
 16  OLIG_PROTE_CD     42

In [7]:
# HCHK_YEAR (기준년도)
df['HCHK_YEAR'].value_counts()

2021    425534
Name: HCHK_YEAR, dtype: int64

모두 '2021' 동일한 값

In [8]:
# IDV_ID (가입자 ID)
df['IDV_ID'].value_counts()

4770160    1
1322733    1
424087     1
4616099    1
2783625    1
          ..
3851115    1
4992030    1
874246     1
4316686    1
4171104    1
Name: IDV_ID, Length: 425534, dtype: int64

모두 unique한 값

In [9]:
# SIDO (시도코드)
df['SIDO'].value_counts()

41    108609
11     75096
48     28470
26     28218
28     24720
47     22666
27     19853
44     17372
46     16482
45     15615
43     13845
42     13099
30     12207
29     11797
31      9472
49      5132
36      2881
Name: SIDO, dtype: int64

In [10]:
# SEX (성별)
df['SEX'].value_counts()

2    221338
1    204196
Name: SEX, dtype: int64

In [11]:
# DATA_STD_DT (데이터 공개일자)
df['DATA_STD_DT'].value_counts()

2022-08-11    425534
Name: DATA_STD_DT, dtype: int64

In [12]:
# 불필요한 데이터 제거
df.drop(['HCHK_YEAR', 'IDV_ID', 'SIDO', 'SEX', 'DATA_STD_DT'], axis=1, inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425534 entries, 0 to 425533
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   AGE_GROUP         425534 non-null  int64  
 1   HEIGHT            425534 non-null  int64  
 2   WEIGHT            425534 non-null  int64  
 3   WAIST             425499 non-null  float64
 4   BP_HIGH           425526 non-null  float64
 5   BP_LWST           425525 non-null  float64
 6   BLDS              425528 non-null  float64
 7   TOT_CHOLE         425058 non-null  float64
 8   TRIGLYCERIDE      425054 non-null  float64
 9   HDL_CHOLE         425054 non-null  float64
 10  LDL_CHOLE         417431 non-null  float64
 11  HMG               425529 non-null  float64
 12  OLIG_PROTE_CD     423059 non-null  float64
 13  CREATININE        425531 non-null  float64
 14  SGOT_AST          425532 non-null  float64
 15  SGPT_ALT          425532 non-null  float64
 16  GAMMA_GTP         42

In [14]:
df.isnull().sum()

AGE_GROUP              0
HEIGHT                 0
WEIGHT                 0
WAIST                 35
BP_HIGH                8
BP_LWST                9
BLDS                   6
TOT_CHOLE            476
TRIGLYCERIDE         480
HDL_CHOLE            480
LDL_CHOLE           8103
HMG                    5
OLIG_PROTE_CD       2475
CREATININE             3
SGOT_AST               2
SGPT_ALT               2
GAMMA_GTP              2
SMK_STAT_TYPE_CD     175
DRK_YN                77
dtype: int64

## **머신 러닝**

In [15]:
# test data 추출 (현재 LDL_CHOLE가 NaN값인 data)
test = df[df['LDL_CHOLE'].isnull()]

In [16]:
# 나머지: train data
train = df.dropna()

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
# LDL_CHOLE (LDL 콜레스테롤 수치 예측)
X = train.drop('LDL_CHOLE', axis=1)
y = train['LDL_CHOLE']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [20]:
X_train.shape, y_train.shape

((277856, 18), (277856,))

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
lg = LinearRegression()

In [23]:
lg.fit(X_train, y_train)

LinearRegression()

In [24]:
# 기울기 & 절편 확인
print(f'''기울기 및 절편 확인
기울기 확인 coef = {lg.coef_}
절편 확인 intercept = {lg.intercept_}''')

기울기 및 절편 확인
기울기 확인 coef = [ 0.0932488   0.01439129  0.0446995   0.05245242 -0.01090481 -0.01039705
 -0.0036746   0.96894635 -0.17346821 -0.7065764   0.11122744 -0.01604032
  0.34978674 -0.00558762  0.00496024 -0.00440681  0.3110144  -0.80840232]
절편 확인 intercept = -23.28838413039017


In [26]:
# 각 가중치(회귀계수) 확인
print("전체에 대하여 가중치 확인")
for index, columns in enumerate(X_train.columns):
  print(f"{columns} = {lg.coef_[index]}")

전체에 대하여 가중치 확인
AGE_GROUP = 0.0932488011147336
HEIGHT = 0.01439129370868375
WEIGHT = 0.04469950352793131
WAIST = 0.05245242345751919
BP_HIGH = -0.010904812890104799
BP_LWST = -0.01039705073458902
BLDS = -0.0036745965423477613
TOT_CHOLE = 0.9689463469678036
TRIGLYCERIDE = -0.17346820921746425
HDL_CHOLE = -0.706576402344796
HMG = 0.1112274413486602
OLIG_PROTE_CD = -0.016040322799394935
CREATININE = 0.3497867412321068
SGOT_AST = -0.005587622687566205
SGPT_ALT = 0.004960236464735231
GAMMA_GTP = -0.004406808687471123
SMK_STAT_TYPE_CD = 0.3110143966565663
DRK_YN = -0.8084023172284575


**TOT_CHOLE, HDL_CHOLE, DRK_YN** 가 영향 많이 줌

In [27]:
# 예측
y_pred = lg.predict(X_test)

In [28]:
# 최종 검증, MSE 사용
from sklearn.metrics import mean_squared_error

In [29]:
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [30]:
print(f'''주요 Feature 삭제 전 rmse = {round(rmse, 3)}''')

주요 Feature 삭제 전 rmse = 7.589


In [31]:
# 가중치 큰 Feature 삭제 (독립 변수의 가중치 변화 확인)
X = X.drop(['TOT_CHOLE', 'HDL_CHOLE', 'DRK_YN'], axis=1)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [33]:
lg2 = LinearRegression()
lg2.fit(X_train, y_train)

LinearRegression()

In [34]:
# 각 가중치(회귀계수) 확인
print(f'''coef
{lg2.coef_}
intercept
{lg2.intercept_}''')

coef
[-2.69729686 -0.35398105  0.04058359 -0.12845854 -0.10944327  0.27192879
 -0.14643456  0.0134492   3.75105938 -1.13494857  0.11085323 -0.02692825
  0.00724149 -0.01037788 -3.15499049]
intercept
175.42572449758666


In [35]:
y_pred = lg2.predict(X_test)

In [37]:
# 각 가중치(회귀계수) 확인
print("가중치 확인")
for index, columns in enumerate(X_train.columns):
  print(f"{columns} = {lg.coef_[index]}")

가중치 확인
AGE_GROUP = 0.0932488011147336
HEIGHT = 0.01439129370868375
WEIGHT = 0.04469950352793131
WAIST = 0.05245242345751919
BP_HIGH = -0.010904812890104799
BP_LWST = -0.01039705073458902
BLDS = -0.0036745965423477613
TRIGLYCERIDE = 0.9689463469678036
HMG = -0.17346820921746425
OLIG_PROTE_CD = -0.706576402344796
CREATININE = 0.1112274413486602
SGOT_AST = -0.016040322799394935
SGPT_ALT = 0.3497867412321068
GAMMA_GTP = -0.005587622687566205
SMK_STAT_TYPE_CD = 0.004960236464735231


In [38]:
# 주요 Feature 삭제 후 RMSE 확인
rmse2 = mean_squared_error(y_test, y_pred, squared=False)

In [39]:
print(f'''주요 Feature 삭제 후 rmse = {round(rmse2, 3)}''')

주요 Feature 삭제 후 rmse = 37.13


Feature의 존재 여부에 따라 rmse 값 매우 크게 차이남

기울기 & 절편 확인으로 어떤 Feature가 얼마나 영향을 주는지 확인